In [1]:
from astropy.io import fits
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

import vice

from astropy.table import Table
import astropy as ap
import astropy.units as u

import surp

In [2]:
file_name = "../../data/allStar-dr17-synspec_rev1.fits.1"

ff = fits.open(file_name, mmap=True)
allStar = ff[1].data
ff.close()

In [3]:
filename = "../../data/apogee_astroNN-DR17.fits"

ff = fits.open(filename, mmap=True)
astroNN = ff[1].data

remove stars with
- STAR_BAD
- NO_ASPCAP_RESULT
- EXTRATARG=0

- R=3-13kpc
- |Z| < 2kpc
- -0.75 < Mg/H < 0.45
- S/N > 200 and Mg/H > -0.5
- SN ? 100 otherwise
- log g = 1-2.5
- Teff = 4000-4600

In [4]:
apogee_aspcapflag = 1<<23 # starbad
apogee_aspcapflag ^= 1<<31 # no_aspcap_result

In [5]:
mask = (allStar["ASPCAPFLAG"] & apogee_aspcapflag) == 0
mask &= allStar["EXTRATARG"] == 0

print(sum(mask))

364344


In [6]:
mg_h = allStar["MG_FE"] + allStar["FE_H"]
mask &= -0.75 <= mg_h
mask &= mg_h <= 0.45
mask &= allStar["SNR"] >= 100
mask &= (mg_h < -0.5)  | (allStar["SNR"] >= 200)

mask &= allStar["LOGG"] >= 1
mask &= allStar["LOGG"] <= 2.5
mask &= allStar["TEFF"] <= 4600
mask &= allStar["TEFF"] >= 4000

print(sum(mask))

29321


In [7]:
filtered = allStar[mask]

In [9]:
df = pd.DataFrame(filtered.tolist(), columns = [c.name for c in filtered.columns])

In [10]:
df_aNN = pd.DataFrame(dict(
    R_gal = astroNN["galr"].byteswap().newbyteorder(),
    z_gal = astroNN["galz"].byteswap().newbyteorder()),
    index = astroNN["APOGEE_ID"].byteswap().newbyteorder())
    



In [11]:
joined = df.join(df_aNN, on="APOGEE_ID", how="left")

In [12]:
len(joined)

31763

In [13]:
len(df)

29321

In [14]:
mask = np.abs(joined["z_gal"]) <= 2
mask &= joined["R_gal"] >= 3
mask &= joined["R_gal"] <= 13

In [16]:
df1 = joined[mask].copy()

df1["MG_H"] = df1["MG_FE"] + df1["FE_H"]
df1["MG_H"] = df1["MG_FE"] + df1["FE_H"]
df1["C_H"] = df1["C_FE"] + df1["FE_H"]
df1["N_H"] = df1["N_FE"] + df1["FE_H"]
df1["C_MG"] = df1["C_H"] - df1["MG_H"]
df1["N_MG"] = df1["N_H"] - df1["MG_H"]
df1["CN_H"] = surp.analysis.cpn(df1["C_H"], df1["N_H"])
df1["CN_MG"] = df1.CN_H - df1.MG_H


df1["high_alpha"] = surp.analysis.is_high_alpha(df1["MG_FE"], df1["FE_H"])

df1.to_csv("../../data/apogee_cartography.csv")

In [17]:
len(df1)

27784